# Analyzing AppStore Data

This project analyzes data fetched from app stores in order to find out most profitable option.

In [1]:
import csv
reader = csv.reader

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
applef = reader(open("AppleStore.csv"))
apple = list(applef)
googlef = reader(open("googleplaystore.csv"))
google = list(googlef)

In [32]:
explore_data(google,2,3, True)
explore_data(apple,2,3, True)


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 8863
Number of columns: 13
['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 3220
Number of columns: 16


We create headers below:

In [5]:
gheader = google[0]
print(gheader)
google = google[1:]
aheader = apple[0]
print(aheader)
apple = apple[1:]

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


## Clear Data
Check if there is any faulty entries

In [6]:
for i in range(len(google)-1):
    if len(google[i]) != len(gheader):
        print("Dropping from google:\n",google[i])
        del google[i]
for i in range(len(apple)):
    if len(apple[i]) != len(aheader):
        print("Dropping from apple:\n",apple[i])
        del apple[i]

Dropping from google:
 ['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


Check if there is  duplicate applications

In [7]:
g_unique_apps = []
g_dup_apps = []
for app in google:
    name = app[0]
    if name in g_unique_apps:
        g_dup_apps.append(name)
    else:
        g_unique_apps.append(name)
print(len(g_unique_apps))
print(len(g_dup_apps))
        
a_unique_apps = []
a_dup_apps = []
for app in apple:
    name = app[1]
    if name in a_unique_apps:
        a_dup_apps.append(name)
    else:
        a_unique_apps.append(name)
print(len(a_unique_apps))
print(len(a_dup_apps))
        

9659
1181
7195
2


Note that we won't drop duplicates randomly. Most recent entry is chosen by functipn below.

In [8]:
g_name_vs_review = {}
for i in google:
    name = i[0]
    cur_rev = float(i[3])
    if name not in g_name_vs_review:
        g_name_vs_review[name] = cur_rev
    else:
        if cur_rev > g_name_vs_review[name]:
            g_name_vs_review[name] = cur_rev
print(len(g_name_vs_review))

a_name_vs_review = {}
for i in apple:
    name = i[1]
    cur_rev = float(i[5])
    if name not in a_name_vs_review:
        a_name_vs_review[name] = cur_rev
    else:
        if cur_rev > a_name_vs_review[name]:
            a_name_vs_review[name] = cur_rev
print(len(a_name_vs_review))

9659
7195


Function below will create clean google list:

In [9]:
google_clean = []
added_apps = []
for app in google:
    name = app[0]
    rev = float(app[3])
    
    if (g_name_vs_review[name] == rev) and (name not in added_apps):
        google_clean.append(app)
        added_apps.append(name)
print(len(google_clean))

apple_clean = []
added_apps = []
for app in apple:
    name = app[1]
    rev = float(app[5])
    
    if (a_name_vs_review[name] == rev) and (name not in added_apps):
        apple_clean.append(app)
        added_apps.append(name)
print(len(apple_clean))

google = google_clean
apple  = apple_clean

9659
7195


Check for if english

In [11]:
def english(name):
    count = 0
    for char in name:
        if ord(char) > 127:
            count += 1
    if count > 3:
        return False
    else:
        return True

google_english = []
apple_english = []

for app in google:
    name = app[0]
    if english(name):
        google_english.append(app)
print(len(google_english))

for app in apple:
    name = app[1]
    if english(name):
        apple_english.append(app)
print(len(apple_english))

google = google_english
apple = apple_english

9614
6181


In [28]:
g_not_free = []
a_not_free = []
g_free = []
a_free = []
for i in range(len(google)):
    name=google[i][0]
    price=google[i][6]
    if price != "Free":
        g_not_free.append(name)
    else:
        g_free.append(google[i])

a_not_free = []
for i in range(len(apple)):
    name=apple[i][1]
    price=apple[i][4]
    if price != "0.0":
        a_not_free.append(name)
    else:
        a_free.append(apple[i])

print(len(g_free), len(a_free))

8863 3220


Reset the datasets:

In [29]:
google = g_free
apple = a_free

Since we want to build an app that succeds in both markets, we would like find out app profiles that are successfull in both markets. To achieve that, we may first look for most common genres:

In [36]:
def genre_freq(genre,dataset):
    if genre in genres[dataset]:
        genres[dataset][genre] += 1
    else:
        genres[dataset][genre] = 1

Function

In [68]:
def freq_table(dataset, index):
    table = {}
    percentage = {}
    for i in range(len(dataset)):
        value = dataset[i][index]
        if value not in table:
            table[value] = 0
        table[value] += 1
    sum = 0
    for count in table:
        sum += table[count]
    for n in table:
        percentage[n] = (table[n]/sum)*100
    return percentage

print(freq_table(google,2))

{'4.1': 6.6794539095114525, '4.7': 4.321335890781902, '4.5': 8.811914701568318, '4.3': 9.522734965587274, '4.4': 9.285794877580955, '3.8': 3.0125239760803337, '4.2': 8.473428861559292, '4.6': 6.8374139681823305, '3.2': 0.6882545413516867, '4.0': 5.517319192147128, '4.8': 2.06476362405506, '3.9': 3.870021437436534, '4.9': 0.8913460453571026, '3.6': 1.771409229380571, '3.7': 2.403249464064087, 'NaN': 14.633871149723571, '3.3': 1.0718718266952498, '3.4': 1.3088119147015682, '3.5': 1.6472977547105947, '3.1': 0.7333859866862237, '5.0': 2.7417353040731127, '3.0': 0.8236488773552973, '2.5': 0.21437436533904997, '2.8': 0.4174658693444658, '2.7': 0.2369400880063184, '1.0': 0.15796005867087895, '1.9': 0.1241114746699763, '2.9': 0.4400315920117342, '2.3': 0.20309150400541578, '2.6': 0.2482229493399526, '2.2': 0.15796005867087895, '1.7': 0.07898002933543948, '2.0': 0.1241114746699763, '2.4': 0.19180864267178158, '1.8': 0.07898002933543948, '1.6': 0.045131445334536835, '2.1': 0.09026289066907367, '

In [69]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [79]:
prime_genre = display_table(apple,11)

Games : 58.13664596273293
Entertainment : 7.888198757763975
Photo & Video : 4.968944099378882
Education : 3.6645962732919255
Social Networking : 3.291925465838509
Shopping : 2.608695652173913
Utilities : 2.515527950310559
Sports : 2.142857142857143
Music : 2.049689440993789
Health & Fitness : 2.018633540372671
Productivity : 1.7391304347826086
Lifestyle : 1.5838509316770186
News : 1.3354037267080745
Travel : 1.2422360248447204
Finance : 1.1180124223602486
Weather : 0.8695652173913043
Food & Drink : 0.8074534161490683
Reference : 0.5590062111801243
Business : 0.5279503105590062
Book : 0.43478260869565216
Navigation : 0.18633540372670807
Medical : 0.18633540372670807
Catalogs : 0.12422360248447205


In [80]:
genres = display_table(google,1)

FAMILY : 18.898792733837304
GAME : 9.725826469592688
TOOLS : 8.462146000225657
BUSINESS : 4.592124562789123
LIFESTYLE : 3.9038700214374367
PRODUCTIVITY : 3.8925871601038025
FINANCE : 3.7007785174320205
MEDICAL : 3.5315355974275078
SPORTS : 3.396141261423897
PERSONALIZATION : 3.317161232088458
COMMUNICATION : 3.2381812027530184
HEALTH_AND_FITNESS : 3.0802211440821394
PHOTOGRAPHY : 2.944826808078529
NEWS_AND_MAGAZINES : 2.798149610741284
SOCIAL : 2.6627552747376737
TRAVEL_AND_LOCAL : 2.335552296062281
SHOPPING : 2.245289405393208
BOOKS_AND_REFERENCE : 2.1437436533904997
DATING : 1.8616721200496444
VIDEO_PLAYERS : 1.7939749520478394
MAPS_AND_NAVIGATION : 1.399074805370642
FOOD_AND_DRINK : 1.241114746699763
EDUCATION : 1.1621347173643235
ENTERTAINMENT : 0.9590432133589079
LIBRARIES_AND_DEMO : 0.9364774906916393
AUTO_AND_VEHICLES : 0.9251946293580051
HOUSE_AND_HOME : 0.8236488773552973
WEATHER : 0.8010831546880289
EVENTS : 0.7108202640189552
PARENTING : 0.6544059573507841
ART_AND_DESIGN : 0

In [81]:
category = display_table(google,9)

Tools : 8.450863138892023
Entertainment : 6.070179397495204
Education : 5.348076272142616
Business : 4.592124562789123
Productivity : 3.8925871601038025
Lifestyle : 3.8925871601038025
Finance : 3.7007785174320205
Medical : 3.5315355974275078
Sports : 3.463838429425702
Personalization : 3.317161232088458
Communication : 3.2381812027530184
Action : 3.102786866749408
Health & Fitness : 3.0802211440821394
Photography : 2.944826808078529
News & Magazines : 2.798149610741284
Social : 2.6627552747376737
Travel & Local : 2.324269434728647
Shopping : 2.245289405393208
Books & Reference : 2.1437436533904997
Simulation : 2.042197901387792
Dating : 1.8616721200496444
Arcade : 1.8503892587160102
Video Players & Editors : 1.771409229380571
Casual : 1.7601263680469368
Maps & Navigation : 1.399074805370642
Food & Drink : 1.241114746699763
Puzzle : 1.128286133363421
Racing : 0.9928917973598104
Role Playing : 0.9364774906916393
Libraries & Demo : 0.9364774906916393
Auto & Vehicles : 0.9251946293580051
S